<center>
    <h1>Verbal Explanation of Spatial Temporal GNNs for Traffic Forecasting</h1>
    <h2>Visualizing an Explained Instance of the PeMS-Bay Dataset</h2>
</center>

---

In this notebook an explained instance of PeMS-Bay is visualized.

In [1]:
import sys
import os

# Set the main path in the root folder of the project.
sys.path.append(os.path.join('..'))

In [2]:
# Settings for autoreloading.
%load_ext autoreload
%autoreload 2

In [3]:
from src.utils.seed import set_random_seed

# Set the random seed for deterministic operations.
SEED = 42
set_random_seed(SEED)

# 1 Loading the Data
The data is loaded.

In [4]:
import os

BASE_DATA_DIR = os.path.join('..', 'data', 'pems-bay')

In [5]:
from src.data.data_extraction import get_adjacency_matrix

# Get the adjacency matrix
adj_matrix_structure = get_adjacency_matrix(
    os.path.join(BASE_DATA_DIR, 'raw', 'adj_mx_pems_bay.pkl'))

# Get the header of the adjacency matrix, the node indices and the
# matrix itself.
header, node_ids_dict, adj_matrix = adj_matrix_structure

In [6]:
from src.data.data_extraction import get_locations_dataframe

# Get the dataframe containing the latitude and longitude of each sensor.
locations_df = get_locations_dataframe(
    os.path.join(BASE_DATA_DIR, 'raw', 'graph_sensor_locations_pems_bay.csv'),
    has_header=False)

In [7]:
# Get the node positions dictionary.
node_pos_dict = { i: id for id, i in node_ids_dict.items() }

In [8]:
import os
import numpy as np

# Get the explained data.
x_test = np.load(os.path.join(BASE_DATA_DIR, 'explained', 'x_test.npy'))
y_test = np.load(os.path.join(BASE_DATA_DIR, 'explained', 'y_test.npy'))

test_clusters = np.load(os.path.join(BASE_DATA_DIR, 'clustered', 'x_test.npy'))
test_translations = np.load(os.path.join(BASE_DATA_DIR, 'translated', 'test.npy'))

# Get the time information of the explained data.
x_test_time = np.load(os.path.join(BASE_DATA_DIR, 'explained', 'x_test_time.npy'))
y_test_time = np.load(os.path.join(BASE_DATA_DIR, 'explained', 'y_test_time.npy'))

In [9]:
# Turn the results in kilometers per hour.
from src.utils.config import MPH_TO_KMH_FACTOR


x_test[..., 0] = x_test[..., 0] * MPH_TO_KMH_FACTOR
y_test = y_test * MPH_TO_KMH_FACTOR

In [10]:
import pickle

with open(os.path.join(BASE_DATA_DIR, 'structured', 'node_locations.pkl'), 'rb') as f:
    node_info = pickle.load(f)

# 2 Visualization
The first instance explanation is visualized.

In [11]:
i = 0

sample_x = x_test[i]
clusters_x = test_clusters[i]
sample_y = y_test[i]
sample_x_time = x_test_time[i]
sample_y_time = y_test_time[i]

clusters_x = clusters_x.astype(object)

verbal_translation = test_translations[i]

In [12]:
_, n_timesteps, n_nodes, _ = y_test.shape

In [13]:
for c in np.unique(clusters_x):
    if c == -1:
        clusters_x[clusters_x == c] = ' '
    else:
        clusters_x[clusters_x == c] = f'cluster {c}'

In [14]:
clusters_y = (sample_y > 0).astype(np.int64).astype(object)
clusters_y[clusters_y == 0] = ' '
clusters_y[clusters_y == 1] = 'target'

In [15]:
from src.explanation.clustering.analyisis import get_node_values_with_clusters_and_location_dataframe

df = get_node_values_with_clusters_and_location_dataframe(sample_x[..., 0:1], clusters_x, node_pos_dict, locations_df, sample_x_time)

In [16]:
icons = {
    ' ': 'cancel',
    'cluster 0': 'star',
    'cluster 1': 'circle',
    'cluster 2':'heart',
    'cluster 3': 'play',
    'cluster 4': 'pause',
    'target': 'certified'}

In [17]:
df['icon'] = df['cluster'].apply(lambda x: icons[x])

In [18]:
print(verbal_translation)

A severe congestion was anticipated on on Bayshore Freeway at km 65 on Thursday, 26/05/2017, with an average speed of 44.81 km/h from 08:20 to 09:15. The motivation was a series of congestions and a free flow.

To begin, a contributing congestion manifested, at 74.35 km/h, on Bayshore Freeway at km 65 from 07:20 to 07:25.

Following this, a contributing severe congestion happened from 07:20 to 07:35 on, another time, Bayshore Freeway at km 65 with an average speed of 50.16 km/h.

Then, a contributing free flow occurred on, yet another time, Bayshore Freeway at km 66, occurring from 07:20 to 08:05 with an average speed of 100.96 km/h.

Then, another contributing congestion happened from 07:20 to 08:10 on, yet another time, Bayshore Freeway at km 66 with an average speed of 95.30 km/h.

In the end, yet an additional contributing congestion happened, with an average speed of 65.39 km/h, on, once again, Bayshore Freeway at km 65 occurring from 07:45 to 08:10.


In [24]:
from src.data.data_analysis import show_kepler_map

# Font size 27
show_kepler_map(
    df, config_file_path='../config/kepler/pems-bay/visualization_test_x_clusters.json')

KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [{'dataId': ['data'], 'id': '00f3xl6z', '…

In [25]:
from src.explanation.clustering.analyisis import (
    get_node_values_with_clusters_and_location_dataframe)

df = get_node_values_with_clusters_and_location_dataframe(sample_y, clusters_y, node_pos_dict, locations_df, sample_y_time)

In [26]:
df['icon'] = df['cluster'].apply(lambda x: icons[x])

In [30]:
from src.data.data_analysis import show_kepler_map

show_kepler_map(
    df, config_file_path='../config/kepler/pems-bay/visualization_test_y_clusters.json')

KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [{'dataId': ['data'], 'id': 'jx1qoqidt', …